# 파일 불러오기

In [3]:
from google.colab import files
myfile = files.upload()

Saving 청와대(전처리).csv to 청와대(전처리).csv


In [4]:
import io
import pandas as pd

In [5]:
data = pd.read_csv(io.BytesIO(myfile['청와대(전처리).csv']))
data.head()

,title,count,categ,start,text
0,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,267,인권/성평등,2020-01-02,"['본인', '서울', '지방', '병무청', '검사장', '탈의실', '발견', ..."
1,주식시장 활성화 및 소액(개미)투자자 보호,271,경제민주화,2020-01-02,"['우리', '나라', '코스', '미국', '애플', '한국', '주식', '시장..."
2,교정기관의 민낮,198,행정,2020-01-02,"['억울', '국민', '신청', '구치소', '교도관', '이주', '구치소', ..."
3,미세먼지 저감 대책,170,안전/환경,2020-01-02,"['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '우..."
4,악질세입자 방지를 위한 세입자보호법을 재정해주세요.,2127,교통/건축/국토,2020-01-02,"['우선', '아이', '부모', '식구', '아이', '성장', '나름', '단꿈..."


In [37]:
# 'text'부분만 추출
president_df = data.loc[:15312, ['text']]
print(president_df)

                                                    text
0      ['본인', '서울', '지방', '병무청', '검사장', '탈의실', '발견', ...
1      ['우리', '나라', '코스', '미국', '애플', '한국', '주식', '시장...
2      ['억울', '국민', '신청', '구치소', '교도관', '이주', '구치소', ...
3      ['미세', '먼지', '심각', '이제', '적극적', '대안', '요구', '우...
4      ['우선', '아이', '부모', '식구', '아이', '성장', '나름', '단꿈...
...                                                  ...
15308  ['충북', '동생', '디스크', '디스크', '가까이', '신경외과', '신경'...
15309  ['안녕', '고등학교', '재학', '중인', '고등학생', '여러', '매체',...
15310  ['대한민국', '대의', '사회', '초년', '이자', '취업', '준비', '...
15311  ['안녕', '사기죄', '무엇', '저희', '엄마', '께서', 'LH', '세...
15312  ['대통령', '간곡히', '제언', '드리', '코로나', '이상', '휩쓸', ...

[15313 rows x 1 columns]


# 역토큰화

In [44]:
president_df.text = president_df.text.str.lstrip("['").str.rstrip("']").str.split("', '").str.join(" ")
print(president_df.text)

0        본인 서울 지방 병무청 검사장 탈의실 발견 탈의실 천장 CCTV 발견 본인 경악 탈...
1        우리 나라 코스 미국 애플 한국 주식 시장 평가 시장 하지만 투자 매력 없다 이렇게...
2        억울 국민 신청 구치소 교도관 이주 구치소 거실 성북구 종암동 구역 편입 정신 보건...
3        미세 먼지 심각 이제 적극적 대안 요구 우리 일상 가장 적극적 모든 건축물 외부 스...
4        우선 아이 부모 식구 아이 성장 나름 단꿈 가지 좀더 분양 하지만 아이 학교 문제 ...
                               ...                        
15308    충북 동생 디스크 디스크 가까이 신경외과 신경 차단 주사 대로 정지 이틀 사망 신경...
15309    안녕 고등학교 재학 중인 고등학생 여러 매체 이슈 고민 응시 신청 이자 백신 그리 ...
15310    대한민국 대의 사회 초년 이자 취업 준비 쓰기 정말 생각 고민 공감 직접 으시 피해...
15311    안녕 사기죄 무엇 저희 엄마 께서 LH 세입자 드리 솔직히 엄마 혼자 키우 하나 키...
15312    대통령 간곡히 제언 드리 코로나 이상 휩쓸 자리 이토록 황량 현재 코로나 대한민국 ...
Name: text, Length: 15313, dtype: object


# TF-IDF 행렬 만들기

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(president_df.text.values.astype('U'))
X.shape

(15313, 54255)

# 토픽 모델링

In [40]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components = 28)
svd_model.fit(X)
len(svd_model.components_)

28

In [46]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n = 10):
  for idx, topic in enumerate(components):
    print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(10)) for i in topic.argsort()[:-n-1:-1]])
get_topics(svd_model.components_, terms)

Topic 1: [('아이', 0.1982737485), ('국민', 0.1516341358), ('코로나', 0.1516323105), ('사람', 0.125037675), ('생각', 0.1222740749), ('저희', 0.1144022227), ('정부', 0.1068780626), ('학교', 0.1054812639), ('상황', 0.1050932834), ('마스크', 0.103766564)]
Topic 2: [('아이', 0.5232621456), ('학교', 0.2325944914), ('교사', 0.1799450256), ('학생', 0.1680275087), ('수업', 0.1333038345), ('어린이집', 0.1305604668), ('개학', 0.1241957048), ('아동', 0.1133167121), ('학대', 0.1122968346), ('보육', 0.0998859253)]
Topic 3: [('마스크', 0.3368037892), ('코로나', 0.2570020868), ('학생', 0.1869113138), ('진자', 0.1627400841), ('학교', 0.1554650847), ('개학', 0.151537441), ('수업', 0.1268458682), ('감염', 0.1262364714), ('시험', 0.1184354031), ('등교', 0.099374174)]
Topic 4: [('주택', 0.322252703), ('아이', 0.245375851), ('임대', 0.1949581558), ('학교', 0.177082082), ('아파트', 0.1466020869), ('대출', 0.1446504512), ('분양', 0.1442604218), ('부동산', 0.1413149787), ('학생', 0.127471521), ('수업', 0.1200355804)]
Topic 5: [('학생', 0.3373208476), ('학교', 0.2785987503), ('시험', 0.2269307779), ('수업